In [ ]:
# Parameters: 
search_text <- "wahlen"
aggregation <- "week" # aggregation can be by day ('day'), week ('week') or month('month')
from_date <-'2017-09-01'
to_date <-'2018-12-31'

In [ ]:
library("httr")
library("jsonlite")
library("ggplot2")
library('RJSplot')

## Using API

In [ ]:
base <- "193.175.238.88:8080/api/"
call_politicians <- paste(base,"politicians/twitter/tweets_by/politicians/?text_contains=", search_text, "&from_date=",from_date,"&to_date=",to_date,"&aggregate_by=", aggregation, sep="")
politicians = as.data.frame(fromJSON(content(GET(call_politicians), "text", encoding="UTF-8"), flatten = TRUE))
call_organizations <- paste(base,"organizations/twitter/tweets_by/organizations/?text_contains=", search_text, "&aggregate_by=", aggregation, sep="")
organizations = as.data.frame(fromJSON(content(GET(call_organizations), "text", encoding="UTF-8"), flatten = TRUE))
call_general <- paste(base,"organizations/twitter/general_population/?text_contains=", search_text, "&aggregate_by=", aggregation, sep="")
general = as.data.frame(fromJSON(content(GET(call_general), "text", encoding="UTF-8"), flatten = TRUE))
merged <- Reduce(function(x,y) merge(x = x, y = y, by = "labels"), 
       list(politicians, organizations, general))

In [ ]:
#plotting
options(repr.plot.width=8, repr.plot.height=4)
ggplot(data = merged, mapping = aes(as.Date(labels))) +
  geom_line(aes(y = values.x, color="Politicians", group=response_type.x)) +
  geom_line(aes(y = values.y, color="Organizations", group=response_type.y)) +
  geom_line(aes(y = values, color="General political tweets", group=response_type)) +
  labs(title = "Twitter (Tweets)", x = "", y = "Tweets") +
  theme(axis.text.x = element_text( size = 7, angle = 60, hjust = 0.5, vjust = 0.5),
        axis.text.y = element_text( size = 7), legend.title = element_blank()) +
scale_x_date(date_breaks = "2 weeks") + scale_y_continuous(labels = scales::number_format(big.mark = ""))